## Initialize Terraform

In [59]:
#rm -rf .terraform rm .terraform.lock.hcl terraform.tfstate terraform.tfstate.backup
!terraform init

## View `resource-group.tf`

In [56]:
#!type resource-group.tf #use this on Windows
!cat resource-group.tf

# Creates a resource group in your Azure account.

resource "azurerm_resource_group" "main" {
  name     = var.app_name
  location = var.location
}


## View `container-storage-account.tf`

In [57]:
#!type container-storage-account.tf #use this on Windows
!cat container-storage-account.tf

# Create a storage account and storage container in Azure (for BLoBs, i.e. arbitrary files)

# Note!
# - Resource "azurerm_resource_group.main" with a property "name" is defined in the file "resource-group.tf".
# - The value for "resource_group_name" below is set using property "name" in resource "azurerm_resource_group.main":
#   - resource_group_name = azurerm_resource_group.main.name
# - "name" and "location" below are set from Terraform variables defined in the file "variables.tf".
# - The value for "storage_account_name" below is set using property "name" in resource "azurerm_storage_account.main":
#   - storage_account_name = azurerm_storage_account.main.name

resource "azurerm_storage_account" "main" {
  name                     = var.app_name
  resource_group_name      = azurerm_resource_group.main.name
  location                 = var.location
  account_kind             = "StorageV2"
  account_tier             = "Standard"
  account_replication_type = "LRS"
}

resource "azurer

## View `container-registry.tf`

In [58]:
#!type container-registry.tf #use this on Windows
!cat container-registry.tf

# Creates a container registry in Azure (for Docker images).
# Note!
# - Resource "azurerm_resource_group.main" with a property "name" is defined in the file "resource-group.tf".
# - The value for "resource_group_name" below is set using property "name" in resource "azurerm_resource_group.main":
#   - resource_group_name = azurerm_resource_group.main.name
# - "name" and "location" below are set from Terraform variables defined in the file "variables.tf".


resource "azurerm_container_registry" "main" {
  name                = var.app_name
  resource_group_name = azurerm_resource_group.main.name
  location            = var.location
  admin_enabled       = true
  sku                 = "Basic"
}


## Terraform Apply

In [5]:
!terraform apply -auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_container_registry.main will be created
  + resource "azurerm_container_registry" "main" {
      + admin_enabled                 = true
      + admin_password                = (sensitive value)
      + admin_username                = (known after apply)
      + encryption                    = (known after apply)
      + export_policy_enabled         = true
      + id                            = (known after apply)
      + location                      = "westeurope"
      + login_server                  = (known after apply)
      + name                          = "tsfn14g00"
      + network_rule_bypass_option    = "AzureServices"
      + network_rule_set              = (known after apply)
      + public_network_access_enabled = true
      + resource_group_name           = "tsf

## List Azure Resource Groups

In [6]:
!az group list -o table

Name       Location    Status
---------  ----------  ---------
tsfn14g00  westeurope  Succeeded


## List Azure Storage Accounts

In [8]:
#!az storage account list --resource-group $RESOURCE_GROUP -o table
!az storage account list -o table

AccessTier    AllowBlobPublicAccess    AllowCrossTenantReplication    AllowSharedKeyAccess    CreationTime                      DefaultToOAuthAuthentication    EnableHttpsTrafficOnly    EnableNfsV3    IsHnsEnabled    IsSftpEnabled    Kind       Location    MinimumTlsVersion    Name       PrimaryLocation    ProvisioningState    PublicNetworkAccess    ResourceGroup    StatusOfPrimary
------------  -----------------------  -----------------------------  ----------------------  --------------------------------  ------------------------------  ------------------------  -------------  --------------  ---------------  ---------  ----------  -------------------  ---------  -----------------  -------------------  ---------------------  ---------------  -----------------
Hot           True                     True                           True                    2024-02-03T08:19:17.256509+00:00  False                           True                      False          False           False      

## List Resources in Resource Group `tsfn14g00`

In [9]:
!az resource list --resource-group tsfn14g00 -o table

Name       ResourceGroup    Location    Type                                    Status
---------  ---------------  ----------  --------------------------------------  --------
tsfn14g00  tsfn14g00        westeurope  Microsoft.ContainerRegistry/registries
tsfn14g00  tsfn14g00        westeurope  Microsoft.Storage/storageAccounts


## List Storage Account Access Keys for Storage Account `tsfn14g00`

- Note that we store the Storage Account name and access key in environment variables so we can use them in the notebook
  - `STORAGE_ACCOUNT_NAME`
  - `STORAGE_ACCESS_KEY`
- Note, this code has only been tested on Ubuntu.

In [2]:
#!az storage account keys list --account-name tsfn14g00 --resource-group tsfn14g00 -o table
!az storage account keys list --account-name tsfn14g00 -o table

!export STORAGE_ACCOUNT_NAME=tsfn14g00
!export STORAGE_ACCESS_KEY=$(az storage account keys list --account-name tsfn14g00 --resource-group tsfn14g00 --query [0].value -o tsv)
!echo -e "\nSTORAGE_ACCOUNT_NAME: $STORAGE_ACCOUNT_NAME"
!echo -e "STORAGE_ACCESS_KEY: $STORAGE_ACCESS_KEY"

CreationTime                      KeyName    Permissions    Value
--------------------------------  ---------  -------------  ----------------------------------------------------------------------------------------
2024-02-03T08:19:17.397136+00:00  key1       FULL           wr5ji3c9ceRaQjv3Wsmzgb/NWVeg0YqNNyErNm0v5p/c4oObYGctCC+rpTbzWc4PXanrFCBC7/CZ+AStkg1pZQ==
2024-02-03T08:19:17.397136+00:00  key2       FULL           f3qwpAJv+iGoJMEnzPlZIgb5OtY9c2VgFD9E7WtBqvh0as2Erv2mPWeK6KXPH1aLW3QWz6a/pXLV+ASt1bVlKA==

STORAGE_ACCOUNT_NAME: tsfn14g00
STORAGE_ACCESS_KEY: wr5ji3c9ceRaQjv3Wsmzgb/NWVeg0YqNNyErNm0v5p/c4oObYGctCC+rpTbzWc4PXanrFCBC7/CZ+AStkg1pZQ==


## List Containers in Storage Account `tsfn14g00`

In [15]:
!az storage container list --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Name    Lease Status    Last Modified
------  --------------  -------------------------
videos                  2024-02-03T08:19:40+00:00


## Create Container `videos2` in Storage Container `tsfn14g00`

In [16]:
!az storage container create --name videos2 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Created
---------
True


## List Containers in Storage Container `tsfn14g00`

In [17]:
!az storage container list --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Name     Lease Status    Last Modified
-------  --------------  -------------------------
videos                   2024-02-03T08:19:40+00:00
videos2                  2024-02-03T08:26:01+00:00


## Delete Container `videos2` in Storage Container `tsfn14g00`

In [18]:
!az storage container delete --name videos2 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Deleted
---------
True


## List Containers in Storage Container `tsfn14g00`

In [19]:
!az storage container list --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Name    Lease Status    Last Modified
------  --------------  -------------------------
videos                  2024-02-03T08:19:40+00:00


## List Blobs (Files) in Container `videos` in Storage Container `tsfn14g00`

In [20]:
!az storage blob list --container-name videos --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table 

## List Local Video File

In [26]:
!ls -al video-storage/videos

total 1040
drwxr-xr-x 2 patrick patrick    4096 Feb  3 09:30 .
drwxr-xr-x 4 patrick patrick    4096 Feb  2 21:49 ..
-rw-r--r-- 1 patrick patrick 1055736 Feb  3 09:30 SampleVideo_1280x720_1mb.mp4


## Upload a file to Container `videos` in Storage Container `tsfn14g00`

In [21]:
!az storage blob upload --container-name videos --name SampleVideo_1280x720_1mb.mp4 --file video-storage/videos/SampleVideo_1280x720_1mb.mp4 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Finished[#############################################################]  100.0000%
Client_request_id                     Content_md5               Date                       LastModified               Request_id                            Request_server_encrypted    Version
------------------------------------  ------------------------  -------------------------  -------------------------  ------------------------------------  --------------------------  ----------
3588db10-c26e-11ee-8632-9769618040bb  1Vvd+NYpEIee2fYFUiFJqA==  2024-02-03T08:28:30+00:00  2024-02-03T08:28:30+00:00  51562d73-501e-002b-3e7a-56f3ca000000  True                        2022-11-02


## List Blobs (Files) in Container `videos` in Storage Container `tsfn14g00`

In [22]:
!az storage blob list --container-name videos --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table 

Name                          Blob Type    Blob Tier    Length    Content Type    Last Modified              Snapshot
----------------------------  -----------  -----------  --------  --------------  -------------------------  ----------
SampleVideo_1280x720_1mb.mp4  BlockBlob    Hot          1055736   video/mp4       2024-02-03T08:28:30+00:00


## Delete Local Video File

In [23]:
!rm video-storage/videos/SampleVideo_1280x720_1mb.mp4
!ls -al video-storage/videos

total 8
drwxr-xr-x 2 patrick patrick 4096 Feb  3 09:30 .
drwxr-xr-x 4 patrick patrick 4096 Feb  2 21:49 ..


## Download a file from Container `videos` in Storage Account `tsfn14g00`

In [24]:
!az storage blob download --container-name videos --name SampleVideo_1280x720_1mb.mp4 --file video-storage/videos/SampleVideo_1280x720_1mb.mp4 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

Finished[#############################################################]  100.0000%
Name                          Blob Type    Blob Tier    Length    Content Type    Last Modified              Snapshot
----------------------------  -----------  -----------  --------  --------------  -------------------------  ----------
SampleVideo_1280x720_1mb.mp4  BlockBlob                 1055736   video/mp4       2024-02-03T08:28:30+00:00


## List Local Video File

In [25]:
!ls -al video-storage/videos

total 1040
drwxr-xr-x 2 patrick patrick    4096 Feb  3 09:30 .
drwxr-xr-x 4 patrick patrick    4096 Feb  2 21:49 ..
-rw-r--r-- 1 patrick patrick 1055736 Feb  3 09:30 SampleVideo_1280x720_1mb.mp4


## Delete a file from Container `videos` in Storage Container `tsfn14g00`

In [27]:
!az storage blob delete --container-name videos --name SampleVideo_1280x720_1mb.mp4 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table

## List Blobs (Files) in Container `videos` in Storage Container `tsfn14g00`

In [28]:
!az storage blob list --container-name videos --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table 

## Upload file again to Container `videos` in Storage Container `tsfn14g00`

In [41]:
!az storage blob upload --container-name videos --name SampleVideo_1280x720_1mb.mp4 --file video-storage/videos/SampleVideo_1280x720_1mb.mp4 --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table
!az storage blob list --container-name videos --account-name $STORAGE_ACCOUNT_NAME --account-key $STORAGE_ACCESS_KEY -o table 

Finished[#############################################################]  100.0000%
Client_request_id                     Content_md5               Date                       LastModified               Request_id                            Request_server_encrypted    Version
------------------------------------  ------------------------  -------------------------  -------------------------  ------------------------------------  --------------------------  ----------
65bee378-c274-11ee-8632-9769618040bb  1Vvd+NYpEIee2fYFUiFJqA==  2024-02-03T09:12:48+00:00  2024-02-03T09:12:48+00:00  2e27203b-f01e-0040-6e81-56743e000000  True                        2022-11-02
Name                          Blob Type    Blob Tier    Length    Content Type    Last Modified              Snapshot
----------------------------  -----------  -----------  --------  --------------  -------------------------  ----------
SampleVideo_1280x720_1mb.mp4  BlockBlob    Hot          1055736   video/mp4       2024-02-03T09:1

## List Azure Container Registries

In [29]:
!az acr list -o table

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-03T08:19:18Z  True


## List Images in Container Registry `tsfn14g00 `

In [30]:
!az acr repository list -n tsfn14g00 --top 10 -o table

## Show Information about Azure Container Registry `tsfn14g00`

- Note that we store the Container Registry URL in an environment variable so we can use it in the notebook
  - `CONTAINER_REGISTRY_LOGIN_SERVER`
- Note, this code has only been tested on Ubuntu.

In [43]:
!az acr show -n tsfn14g00 -o table

!export CONTAINER_REGISTRY_LOGIN_SERVER=$(az acr show -n tsfn14g00 --query loginServer | sed 's/^"//;s/"$//')
!echo -e "\nCONTAINER_REGISTRY_LOGIN_SERVER: $CONTAINER_REGISTRY_LOGIN_SERVER"

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-03T08:19:18Z  True

CONTAINER_REGISTRY_LOGIN_SERVER: tsfn14g00.azurecr.io


## Show Credentials for Azure Container Registry `tsfn14g00`

- Note that we store the Container Registry Username and Password in environment variables so we can use them in the notebook
  - `CONTAINER_REGISTRY_USERNAME`
  - `CONTAINER_REGISTRY_PASSWORD`
- Note, this code has only been tested on Ubuntu.

In [44]:
!az acr credential show -n tsfn14g00 -o table

!export CONTAINER_REGISTRY_USERNAME=$(az acr credential show -n tsfn14g00 --query username | sed 's/^"//;s/"$//')
!export CONTAINER_REGISTRY_PASSWORD=$(az acr credential show -n tsfn14g00 --query passwords[0].value | sed 's/^"//;s/"$//')
!echo -e "\nCONTAINER_REGISTRY_USERNAME: $CONTAINER_REGISTRY_USERNAME"
!echo "CONTAINER_REGISTRY_PASSWORD: $CONTAINER_REGISTRY_PASSWORD"

USERNAME    PASSWORD                                              PASSWORD2
----------  ----------------------------------------------------  ----------------------------------------------------
tsfn14g00   z7yOQz9XYiBJDVBULymhYbo4j43m3PYv8r208xjdwD+ACRBe6j9B  LJCLDvMK2h7BTupHezY9ThxHW8jaN4qqjNo0GDiyn9+ACRCZION9

CONTAINER_REGISTRY_USERNAME: tsfn14g00
CONTAINER_REGISTRY_PASSWORD: z7yOQz9XYiBJDVBULymhYbo4j43m3PYv8r208xjdwD+ACRBe6j9B


## Login to Azure Container Registry via Docker

In [33]:
!echo $CONTAINER_REGISTRY_PASSWORD | docker login $CONTAINER_REGISTRY_LOGIN_SERVER -u $CONTAINER_REGISTRY_USERNAME --password-stdin  > /dev/null 2>&1

## Build and Push Docker Image to Azure Container Registry

- Note, this code has only been tested on Ubuntu.

In [34]:
# Prepare Nodejs Application
#rm -rf ./video-storage/node_modules ./video-storage/package.json ./video-storage/package-lock.json
cd ./video-storage && \
npm init -y && \
npm pkg set 'main'='./src/index.js' && \
npm pkg set 'scripts.start'='node ./src/index.js' && \
npm pkg set 'scripts.start:dev'='nodemon ./src/index.js' && \
npm pkg delete 'scripts.test' && \
npm install --silent --save express@5.0.0-beta.1 && \
npm install --silent --save @azure/storage-blob@12.17.0 && \
npm install --silent --save-dev nodemon@3.0.3 && \
cd ..

# Build Docker image with Nodejs Application
docker build -q -t video-storage:1 -f ./video-storage/Dockerfile ./video-storage
docker tag video-storage:1 $CONTAINER_REGISTRY_LOGIN_SERVER/video-storage:1

# Push Docker Image to Azure Container Registry
docker push $CONTAINER_REGISTRY_LOGIN_SERVER/video-storage:1

# Clean up
docker rmi video-storage:1 > /dev/null 2>&1
docker rmi $CONTAINER_REGISTRY_LOGIN_SERVER/video-storage:1 > /dev/null 2>&1
docker image list | grep -i -E 'REPOSITORY|video-storage'

Wrote to /home/patrick/projects/tsfn14/05_storage_account/video-storage/package.json:

{
  "name": "video-storage",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}


(##################) ⠼ reify:nodemon: timing reifyNode:node_modules/semver Comppdm[?25h(⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂) ⠦ idealTree:video-storage: sill idealTree buildDeps(⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂) ⠦ idealTree:video-storage: sill idealTree buildDeps
failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

sha256:21f7d21166bc82955537727e0ceabedc98194996be3a04c2747b5c387f4e76cf
The push refers to repository [tsfn14g00.azurecr.io/video-stor

## List Images in Container Registry `tsfn14g00 `

In [35]:
!az acr repository list -n tsfn14g00 --top 10 -o table

Result
-------------
video-storage


## Show Info. about Image `video-storage` in Azure Container Registry

In [36]:
!az acr repository show -n tsfn14g00 --image video-storage:1 -o table
!echo ''
!az acr repository show -n tsfn14g00 --repository video-storage -o table
!echo ''
!az acr repository show-tags -n tsfn14g00 --repository video-storage --top 10 -o table

CreatedTime                   Digest                                                                   LastUpdateTime                Name    Signed
----------------------------  -----------------------------------------------------------------------  ----------------------------  ------  --------
2024-02-03T08:58:08.4789932Z  sha256:65bac33804523d4ad30423d7c871e438511f8788ec9f9b12324e75f9f527cb0e  2024-02-03T08:58:08.4789932Z  1       False

CreatedTime                  ImageName      LastUpdateTime                ManifestCount    Registry              TagCount
---------------------------  -------------  ----------------------------  ---------------  --------------------  ----------
2024-02-03T08:58:08.315909Z  video-storage  2024-02-03T08:58:08.4220751Z  1                tsfn14g00.azurecr.io  1

Result
--------
1


## Show Azure Container Registry Usage

In [37]:
!az acr show-usage -n tsfn14g00 -o table

NAME       LIMIT        CURRENT VALUE    UNIT
---------  -----------  ---------------  ------
Size       10737418240  60569775         Bytes
Webhooks   2            0                Count
ScopeMaps  100          0                Count
Tokens     100          0                Count


## Pull and Run Docker Image from Azure Container Registry

- Note, this code has only been tested on Ubuntu.

In [45]:
docker run --name video-storage -d -p 3000:3000 \
           -e PORT=3000 \
           -e STORAGE_ACCOUNT_NAME=$STORAGE_ACCOUNT_NAME \
           -e STORAGE_ACCESS_KEY=$STORAGE_ACCESS_KEY \
           $CONTAINER_REGISTRY_LOGIN_SERVER/video-storage:1

docker ps
docker logs video-storage
#docker exec -it video-storage /bin/sh

# Open URL `http://localhost:3000/video` in Firefox browser
firefox http://localhost:3000/video?path=SampleVideo_1280x720_1mb.mp4

a432c68649855a7f0beee4797f72c81e6ab1df07a4c75baaf40d27ff77ce71ee
CONTAINER ID   IMAGE                                  COMMAND                  CREATED                  STATUS                  PORTS                                       NAMES
a432c6864985   tsfn14g00.azurecr.io/video-storage:1   "docker-entrypoint.s…"   Less than a second ago   Up Less than a second   0.0.0.0:3000->3000/tcp, :::3000->3000/tcp   video-storage
[GFX1-]: glxtest: ManageChildProcess failed



## Stop and Remove Container and Local Image

In [46]:
!docker stop video-storage
!docker rm video-storage
!docker ps -a | grep -i -E 'IMAGE|video-storage'
!docker rmi $CONTAINER_REGISTRY_LOGIN_SERVER/video-storage:1 > /dev/null 2>&1
!docker image list | grep -i -E 'REPOSITORY|video-storage'

video-storage
video-storage
CONTAINER ID   IMAGE                                 COMMAND                  CREATED      STATUS                    PORTS                                                                                                                                  NAMES
REPOSITORY                    TAG             IMAGE ID       CREATED         SIZE


## Logout from Azure Container Registry via Docker

In [47]:
!docker logout $CONTAINER_REGISTRY_LOGIN_SERVER > /dev/null 2>&1

## Delete Image `video-storage` from Azure Container Registry

In [48]:
#!az acr repository delete -y -n tsfn14g00 --image video-storage:1 -table
!az acr repository delete -y -n tsfn14g00 --repository video-storage -o table

## List Images in Container Registry `tsfn14g00 `

In [49]:
!az acr repository list -n tsfn14g00 --top 10 -o table

## Terraform Destroy

In [50]:
!terraform destroy -auto-approve

azurerm_resource_group.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]
azurerm_container_registry.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.ContainerRegistry/registries/tsfn14g00]
azurerm_storage_account.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.Storage/storageAccounts/tsfn14g00]
azurerm_storage_container.main: Refreshing state... [id=https://tsfn14g00.blob.core.windows.net/videos]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # azurerm_container_registry.main will be destroyed
  - resource "azurerm_container_registry" "main" {
      - admin_enabled                 = true -> null
      - admin_password     

## List Azure Container Registries

In [51]:
!az acr list -o table

## List Azure Storage Accounts

In [52]:
#!az storage account list --resource-group $RESOURCE_GROUP -o table
!az storage account list -o table

## List Azure Resource Groups

In [53]:
!az group list -o table